In [ ]:
import warnings
warnings.simplefilter("ignore")

### open questions

1. Evaluation
    - Can we just accept data leakage? Is it acceptable to only be able to predict tomorrow’s closing in practice? They do in the paper it seems.
2. Target
    - Is close price a good target or should we try to predict trading signal (whatever the fuck that is)?
3. Technicals
    - There doesn’t seem to be much of a rhyme or reason to the technicals they chose. I’ll try to adapt the ones we have to match the paper as well as possible.


## Technicals
------

**Approach:** Sift through technical functions in `ta.py` file, research sensible parameters for each function. Sometimes it seems like multiple runs should be extracted with each function with different parameters, which is denoted by a list of values. 

### Good to go

- 'MA'
    - n: [5, 20, 90, 260]
- 'STDDEV'
    - n: [5, 20, 90, 260]
- 'RSI'
    - n: [6, 12]
- MACD'
    - n_fast: 12
    - n_slow: 26

- 'BBANDS'
    - n: [5, 20, 90, 260]

- 'MFI' money flow index ratio
    - n: 14
- 'Chaikin'
    - None
- 'EMA'
    - n: [5, 20, 90, 260]
- 'KST'
    - r: (10, 10, 10, 15)
    - n: (10, 15, 20, 30)
    
- 'TSI'
    - r: 25
    - s: 13

- 'TRIX'
    - n: [5, 20, 90, 260]

- 'STOK'
    - None

- 'STO'
    - n: [5, 20, 90, 260]

- 'ROC'
    - n: [5, 20, 90, 260]

- 'PPSR' 

- 'OBV'
    - n: [5, 20, 90, 260]

- 'MassI'
    - None
    
- 'MOM'
    - n: 1 

- 'COPP' 
    - n: 10

- ACCDIST'
    - n: 1

- 'ADX'
    - n: 14
    - n_ADX: 50
- 'ATR'
    - n: 14

### Potential implementation

- differences on any or all of these columns

### Missing end data

'ULTOSC'

'Vortex'

'EOM' ease of movement

'KELCH'

'DONCH'

'CCI' Commodity channel index

In [ ]:
cd .. 

In [ ]:
#load technicals .py file
from lib import ta
import pandas as pd
import inspect
import string
import os

In [ ]:
#hacky way to get a dictionary of all the imported technical functions
tech_funcs = dict(filter(lambda x: x[0][0] in string.ascii_uppercase, inspect.getmembers(ta)))

In [ ]:
#Each tuple contains the *args for a single run
#Best guesses from the internet, "A critical extraction .." paper, and the ta.py code

grid = {"MA": [(5,), (20,), (90,), (260,)],
        "STDDEV": [(5,), (20,), (90,), (260,)],
        "RSI": [(6,), (12,)],
        "MACD": [(12, 26)],
        "BBANDS": [(5,), (20,), (90,), (260,)],
        "MFI": [(14,)],
        "Chaikin": [()],
        "EMA": [(5,), (20,), (90,), (260,)],
        "KST": [(10, 10, 10, 15, 10, 15, 20, 30)],
        "TSI": [(25, 13)],
        "TRIX": [(5,), (20,), (90,), (260,)],
        "STOK": [()],
        "STO": [(5,), (20,), (90,), (260,)],
        "ROC": [(5,), (20,), (90,), (260,)],
        "PPSR": [()],
        "OBV": [(5,), (20,), (90,), (260,)],
        "MassI": [()],
        "MOM": [(1,)],
        "COPP": [(10,)],
        "ADX": [(14, 50)],
        "ATR": [(14,)],
        "FORCE": [(2,)],
        "ACCDIST": [(1,)]}


In [ ]:
#serially apply every technical function in dictionary to an initial dataframe
def extract_technicals(df, tech_funcs, grid):
    
    output = df
    for name, func in tech_funcs.items():
        arg_list = grid[name]
        for arg_tuple in arg_list:
            output = func(output, *arg_tuple)
    
    return output

### Extract technicals from every individual stock CSV

In [ ]:
import os

In [ ]:
ORIGINAL_DIR = "data/sandp500/individual_stocks_5yr/"
TECHNICALS_DIR = "data/sandp500/individual_stocks_5yr_TECHNICALS/"
try:
    os.stat(TECHNICALS_DIR)
except:
    os.mkdir(TECHNICALS_DIR)  

In [ ]:
#grab list of csv names in the directory
individuals = os.listdir(path = "data/sandp500/individual_stocks_5yr/")
DEBUG = False

for csv in individuals:
    if csv != '.DS_Store':
        csv_path = ORIGINAL_DIR + csv
        if DEBUG: print(csv)
        df = pd.read_csv(csv_path)
        
        with open(TECHNICALS_DIR+csv, "w+") as file:
            try:
                df_technicals = extract_technicals(df, tech_funcs, grid)
                df_technicals.to_csv(file)
            except IndexError:
                print(f"Technical extraction failed on {csv}")


### Testing `extract_technicals` on single stock csv

In [ ]:
sp = pd.read_csv("data/sandp500/individual_stocks_5yr/A_data.csv")
sp_technicals = extract_technicals(sp, tech_funcs, grid)

### Notes

- `ACCDIST` appears to be spitting out all zeros.
- Barely any variance in `TRIX` and `TRIX_260` is about half missing. Feel like the args for this function are probably wrong.